### HW1 Создание и нормализация базы данных

### Задачи

* Продумать структуру БД и отрисовать в редакторе; (вып.)
* Нормализовать БД (1НФ-3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась;
* Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет;
* Загрузить данные в таблицы в соответствии с созданной структурой.

## Решение

### Проанализируем исходный датасет

Исходный датасет представлени в виде документа в формате .xlsx, в котором содержится две таблицы: 'transaction' и 'customer', следующего вида:

transaction:

![transaction](https://thumb.cloud.mail.ru/weblink/thumb/xw1/hwck/PjwBBzZuP)

customer:

![customer](https://thumb.cloud.mail.ru/weblink/thumb/xw1/6X6S/d5XJvEJmN)

Глядя на талицы можно выделить четыре основные сущности: transaction, customer, product и address (адреса лучше выделить в отдельный справочник так как у одного покупателя может быть несколько адресов доставки)

Разделим таблицы по сущностям с помощью pandas, но сначала выделим признаки для каждой сущности:

* transaction: 

![transaction](https://thumb.cloud.mail.ru/weblink/thumb/xw1/KhME/TgaiUXQPE)

* customer:

![customer](https://thumb.cloud.mail.ru/weblink/thumb/xw1/xHRs/hdKoDUXXh)

* product:

![product](https://thumb.cloud.mail.ru/weblink/thumb/xw1/AhFT/ss4GmQHTp)


* address:

![address](https://thumb.cloud.mail.ru/weblink/thumb/xw1/nKBJ/hVQnbsPqk)


Колонки list_price и standard_cost оставляем в transaction и в product так как нам необходимо в транзакциях видеть на момент совершения покупки, а в product мы будем видеть актуальные цены на продукт

In [1]:
import pandas as pd
import openpyxl

Загрузим таблицу transaction:

In [2]:
transactions_df = pd.read_excel('HW1_data.xlsx')
transactions_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [3]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      19640 non-null  object        
 5   order_status      20000 non-null  object        
 6   brand             19803 non-null  object        
 7   product_line      19803 non-null  object        
 8   product_class     19803 non-null  object        
 9   product_size      19803 non-null  object        
 10  list_price        20000 non-null  float64       
 11  standard_cost     19803 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.8+ MB


Загрузим таблицу customer:

In [4]:
customers_df = pd.read_excel('HW1_data.xlsx', 1)
customers_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [5]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

### Разделим две исходные таблицы на 4 датасета по сущностям, которые мы выделили раньше:

In [6]:
transaction_df = transactions_df[['transaction_id', 'product_id', 'customer_id', 'transaction_date', 'online_order', 'order_status', 'list_price', 'standard_cost']]
transaction_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,1765.30,709.48


In [7]:
customer_df = customers_df[['customer_id', 'first_name', 'last_name', 'gender', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car']]
customer_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes


In [8]:
product_df = transactions_df[['product_id', 'brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost']]
product_df.head()

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
0,2,Solex,Standard,medium,medium,71.49,53.62
1,3,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,37,OHM Cycles,Standard,low,medium,1793.43,248.82
3,88,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,78,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [9]:
address_df = customers_df[['address', 'postcode', 'state', 'country', 'property_valuation']]
address_df.head()

,address,postcode,state,country,property_valuation
0,060 Morning Avenue,2016,New South Wales,Australia,10
1,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,0 Holy Cross Court,4211,QLD,Australia,9
3,17979 Del Mar Point,2448,New South Wales,Australia,4
4,9 Oakridge Court,3216,VIC,Australia,9


### Разбираемся с ключами для каждой таблицы

## Transaction

Проверим transaction_id на пропуски и дубликаты:

In [10]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      19640 non-null  object        
 5   order_status      20000 non-null  object        
 6   list_price        20000 non-null  float64       
 7   standard_cost     19803 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.2+ MB


In [11]:
duplicate_transaction = transaction_df.duplicated(['transaction_id'])
duplicate_transaction.value_counts()

False    20000
Name: count, dtype: int64

Так как transaction_id не имеет дубликатов и пропусков принимаем данный признак за первичный ключ для таблицы transaction

Проверим customer_id на пропуски и дубликаты:

In [12]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
dtypes: int64(1), object(9)
memory usage: 312.6+ KB


In [13]:
duplicate_customer = customer_df.duplicated(['customer_id'])
duplicate_customer.value_counts()

False    4000
Name: count, dtype: int64

Так как customer_id не имеет дубликатов и пропусков принимаем данный признак за первичный ключ для таблицы customer

Проверим product_id на пропуски и дубликаты:

In [14]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     20000 non-null  int64  
 1   brand          19803 non-null  object 
 2   product_line   19803 non-null  object 
 3   product_class  19803 non-null  object 
 4   product_size   19803 non-null  object 
 5   list_price     20000 non-null  float64
 6   standard_cost  19803 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB


In [15]:
duplicate_product = product_df.duplicated(['product_id'])
duplicate_product.value_counts()

True     19899
False      101
Name: count, dtype: int64

Так как product_id имеет много дубликатов мы не можем взять его как первичный ключ для таблицы product.
Попробуем удалить дубликаты в product_df

In [16]:
product_df_copy = product_df.drop_duplicates()

In [17]:
duplicate_product_copy = product_df_copy.duplicated(['product_id'])
duplicate_product_copy.value_counts()

True     298
False    101
Name: count, dtype: int64

In [18]:
product_df_copy.sample(10)

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
3194,0,NaN,NaN,NaN,NaN,638.29,NaN
475,21,WeareA2B,Touring,medium,medium,1466.68,363.25
60,0,OHM Cycles,Road,high,large,12.01,7.21
1732,0,Solex,Standard,medium,large,202.62,151.96
84,84,Trek Bicycles,Road,medium,medium,290.62,215.14
58,93,OHM Cycles,Standard,high,medium,1458.17,874.90
12968,0,NaN,NaN,NaN,NaN,487.80,NaN
149,9,OHM Cycles,Road,medium,medium,742.54,667.40
34,0,Norco Bicycles,Road,medium,medium,544.05,376.84
16681,0,NaN,NaN,NaN,NaN,877.44,NaN


In [19]:
product_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399 entries, 0 to 19871
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     399 non-null    int64  
 1   brand          203 non-null    object 
 2   product_line   203 non-null    object 
 3   product_class  203 non-null    object 
 4   product_size   203 non-null    object 
 5   list_price     399 non-null    float64
 6   standard_cost  203 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.9+ KB


Удалив дубликаты в датасете мы видим что для product_id	== 0 осталось очень много аномальных записей (отсутсвуют 5 из 6 признаков), данный датасет требует более детальной проработки так как аномальные записи не несут информативноссти для БД (но это не входит в данное ДЗ). Просто удалим строки где отсутствуют 5 из 6 признаков.

In [20]:
product_df_copy = product_df_copy[product_df_copy['brand'].notna()]


In [21]:
product_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 0 to 17468
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     203 non-null    int64  
 1   brand          203 non-null    object 
 2   product_line   203 non-null    object 
 3   product_class  203 non-null    object 
 4   product_size   203 non-null    object 
 5   list_price     203 non-null    float64
 6   standard_cost  203 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 12.7+ KB


Итого в таблице product 203 строки с уникальными значениями

На основании анализа принимаем решение добавить для product_df суррогатный первичный ключ:

In [22]:
product_df = product_df_copy.sort_values('product_id').reset_index(drop=True)

In [23]:
product_df = product_df.reset_index().rename(columns={'index': 'product_table_id'})
product_df.head()

,product_table_id,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
0,0,0,OHM Cycles,Road,medium,medium,742.54,667.400024
1,1,0,Norco Bicycles,Road,medium,medium,544.05,376.840000
2,2,0,Trek Bicycles,Standard,medium,medium,499.53,388.720000
3,3,0,OHM Cycles,Standard,medium,medium,235.63,125.070000
4,4,0,Solex,Standard,medium,medium,478.16,298.720000


In [24]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_table_id  203 non-null    int64  
 1   product_id        203 non-null    int64  
 2   brand             203 non-null    object 
 3   product_line      203 non-null    object 
 4   product_class     203 non-null    object 
 5   product_size      203 non-null    object 
 6   list_price        203 non-null    float64
 7   standard_cost     203 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 12.8+ KB


## Address

В таблице adress_df изначально нет признака, который мы можем взять за первичный ключ. Поэтому разу сделаем суррогатный ключ.

In [25]:
address_df = address_df.reset_index().rename(columns={'index': 'address_id'})
address_df.head()

,address_id,address,postcode,state,country,property_valuation
0,0,060 Morning Avenue,2016,New South Wales,Australia,10
1,1,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,2,0 Holy Cross Court,4211,QLD,Australia,9
3,3,17979 Del Mar Point,2448,New South Wales,Australia,4
4,4,9 Oakridge Court,3216,VIC,Australia,9


In [26]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   address_id          4000 non-null   int64 
 1   address             4000 non-null   object
 2   postcode            4000 non-null   int64 
 3   state               4000 non-null   object
 4   country             4000 non-null   object
 5   property_valuation  4000 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [27]:
duplicate_address = address_df.duplicated(['address_id'])
duplicate_address.value_counts()

False    4000
Name: count, dtype: int64

В колонке address указан полный адрес (дом и улица). Имеет смысл сразу разделить данный признак на два, так как в будущем это нужно будет сделать для нормализации БД.

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
address_df['address'] = address_df['address'].map(lambda x: x.split(' ', 1))

In [30]:
address_df['house_number'] = address_df['address'].map(lambda x: x[0])

In [31]:
address_df['street'] = address_df['address'].map(lambda x: x[1])

In [32]:
address_df = address_df.drop('address', axis=1)

In [33]:
address_df

,address_id,postcode,state,country,property_valuation,house_number,street
0,0,2016,New South Wales,Australia,10,060,Morning Avenue
1,1,2153,New South Wales,Australia,10,6,Meadow Vale Court
2,2,4211,QLD,Australia,9,0,Holy Cross Court
3,3,2448,New South Wales,Australia,4,17979,Del Mar Point
4,4,3216,VIC,Australia,9,9,Oakridge Court
...,...,...,...,...,...,...,...
3995,3995,4511,QLD,Australia,6,57042,Village Green Point
3996,3996,2756,NSW,Australia,10,87,Crescent Oaks Alley
3997,3997,4032,QLD,Australia,7,8194,Lien Street
3998,3998,2251,NSW,Australia,7,320,Acker Drive


Готово! Этого хватит чтобы выполнить первое задание, проектирование БД в редакторе.

### Проектируем БД в редакторе

Для проектирования используем dbdiagram.io

![dbdiagram](https://thumb.cloud.mail.ru/weblink/thumb/xw1/PPBP/vyHy1cRVU)

Скрипт:

![skript1](https://thumb.cloud.mail.ru/weblink/thumb/xw1/hbtN/pWCskGxFr)
![skript2](https://thumb.cloud.mail.ru/weblink/thumb/xw1/Dngh/Rv6ForN24)

# Нормализация

Берем исходные датасеты

In [34]:
transactions_df = pd.read_excel('HW1_data.xlsx')
transactions_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


In [35]:
customers_df = pd.read_excel('HW1_data.xlsx', 1)
customers_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


### Ненормализованная форма (нулевая нормальная форма)

Исходные таблицы не нарушают базовых принципов реляционной теории - значит можно приступать к нормализации!
(порядок строк и столбцов не имеет значения и не влияет на хранимые данные)

Вывод: таблицы изначально находятся в нулевой нормальной форме

### Первая нормальная форма

Требования:

* Таблицы находятся в нулевой нормальной форме;
* В таблице не должно быть дубрирующихся строк;
* В каждой ячейке хранится атомарное значение (не составное);
* В столбцах хранятся данные 1 типа;
* Отсутствуют массивы и списки в любом виде.

Для начала удалим дубликаты из таблиц, если они там есть

In [36]:
transactions_df.drop_duplicates()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19996,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74
19997,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19998,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73


In [37]:
customers_df.drop_duplicates()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09 00:00:00,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
3996,3997,Blanch,Nisuis,Female,2001-07-13 00:00:00,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,NaN,Male,1973-10-24 00:00:00,NaN,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7
